Logic for parsing chembl dataset to curate effector gene list
Load chembl dataset
filter by phase 3+4
keep efo-gene pairings which were successful

Load L2G feature matrix
Load study index
Load credible sets # maybe not needed

For each efo-gene pairing:
    Identify studylocusId-to-gene pairings that match the efo
    Select the entry with the least amount of feature missingness

Filter entries based on feature strength, do not want entries with 1 feature or 2 features only.
    Investigate, how many entries remain after different thresholds? at least 5/4/3 features?

This will be the new effector gene list / gold standard positives

For the negatives:
Join the studylocusId in the entries back to the credible set.
    Using the variantId information in the credible set, join with the gene index to identify other nearby genes.
    These will become gold standard negatives.
    Recycle the logic from current GSN generation to filter using stringDB < 0.7

    Furthermore, apply a similar filter on the GSN feature matrix, retaining only top 1/2/3/4/5 entries with at least 3/4/5 features.

Investigate the schema required for the gold standards
Annotate the GSP + GSN entries accordingly.
Modify the code in l2g.py to utilise GSP + GSN input directly, instead of automatically identifying GSNs each run.


